Для самого меньшего из текстов выбранного писателя произвести поиск аналогий по типу "Мужчина-Женщина -> Король-Королева" (пары слов, соединяемые векторами одного направления и близких по абсолютной величине). В поиске допускается ограничиться 100 слов.

**Приложение к работе:** <br>
В работе есть несколько отклонений от поставленной задачи: <br>
Поиск аналогий произведён не по одному тексту писателя, а по всем шести (сгруппированы 1,2 и 3 произведения и 4,5 и 6 произведения) <br>
Обоснование для данного отклонения: <br>
Поскольку данные, на которых изначально работает модель word2vec, не имеют спецификации Бунинских произведений, они показывали бы векторы не относительно данного корпуса текстов, а относительно тех, на которых модель была обучена. Поэтому я решила, что было бы логичнее обучить модель на текстах Бунина (для обучения данных самого маленького произведения бы не хватило). В этом случае имеет место и поиск аналогий произвести не по одному произведению, а по всем. <br>

Для данной задачи я составила следующий фреймворк работы: <br>
1) Задать функции для <br>
    А) Обработки текста для получения самых часто-используемых слов <br>
    Б) Обработки текста для модели <br>
2) Применить функцию А на текст, получить список слов <br>
3) Натренировать модель <br>
4) Задать функцию для вывода аналогий по шести сгруппированным текстам <br>
5) Применить функцию <br>


In [ ]:
%pip install gensim

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader

word2vec = gensim.downloader.load('word2vec-ruscorpora-300')

In [ ]:
import codecs
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pymorphy3
import spacy
import os

morph = pymorphy3.MorphAnalyzer()
stop = stopwords.words("russian")


In [ ]:
nlp = spacy.load('ru_core_news_sm')
nlp.add_pipe('sentencizer')

Расширяю список стоп-слов (тех не хватило)

In [ ]:
to_add = ["это", "который", "ещё", "всё","аж","точно","наш","очень","весь", "около", "кой", "таки", "хотя"]
stop.extend(to_add)

In [ ]:
def optokfil(filename): 
        with codecs.open(filename, "r", "utf-8") as my_file:
            text = my_file.read().lower()
        tokenized = RegexpTokenizer(r'\w+').tokenize(text)

        lemmatized = []
        for word in tokenized:
            word = morph.parse(word)[0].normal_form
            lemmatized.append(word)
        
        words_filtered = [word for word in lemmatized if word not in stop]
        
        return words_filtered

In [ ]:
def cooker(text): #makes food for word2vec
        with codecs.open(text, "r", "utf-8") as my_file:
                food = my_file.read()

        food_nlp = nlp(food)

        sent_food = []
        for sentence in food_nlp.sents:
               sent_food.append([sentence])

        regexp_food = []
        for sentence in sent_food:
                final_sent = RegexpTokenizer(r'\w+').tokenize(str(sentence))
                regexp_food.append(final_sent)

        final_food = []
        for sentence in regexp_food:
                sent = []
                for word in sentence:
                        w = morph.parse(word)[0].normal_form
                        sent.append(w)
                final_food.append(sent)
        return final_food

In [68]:
def most_common(text):
    d = {}
    for word in text:
        d[word] = text.count(word)

    return sorted(d.items(),key=lambda item: item[1], reverse=True)[0:53]

In [69]:
mc_dict = {} #mostcommon dictionary
for filename in os.scandir("6_texts"):
        mc_dict["{}".format(filename.name)] = most_common(optokfil(filename.path))

#это чтобы брать частые слова

In [ ]:
massive_food = [] #food for our model
for filename in os.scandir("6_texts"):
        massive_food.extend(cooker(filename.path))

#это чтобы кормить модель

In [ ]:
model = Word2Vec(massive_food, vector_size=32, epochs=100)
word_vectors = model.wv

In [72]:
def search(dict):
    
    counter = 1

    for num in range(50):
        p1 = (mc_dict["1_anton_yabl_tok_1895.txt"][num][0])
        p2 = (mc_dict["2_suhodol_1909_tok.txt"][num][0])
        n = (mc_dict["3_nochnoy_razg_1911_tok.txt"][num][0])
        print (f"{p1} + {p2} - {n} = {word_vectors.most_similar(positive=[p1, p2], negative=[n], topn=1)[0]} \n")
        if p1 not in word_vectors.index_to_key or p2 not in word_vectors.index_to_key or n not in word_vectors.index_to_key:
            continue
        counter +=1

    for num in range(50):
        p1 = (mc_dict["4_gospodin_iz_sanfran_tok_1915.txt"][num][0])
        p2 = (mc_dict["5_zhizn_arsen_1928_tok.txt"][num][0])
        n = (mc_dict["6_pozniy_chas_1938_tok.txt"][num][0])
        if p1 not in word_vectors.index_to_key or p2 not in word_vectors.index_to_key or n not in word_vectors.index_to_key:
            continue
        print (f"{p1} + {p2} - {n} = {word_vectors.most_similar(positive=[p1, p2], negative=[n], topn=1)[0]} \n")
        counter +=1

    print (f"Search выполнил {counter} строк и пошёл спать")
            

In [74]:
search(mc_dict)

сад + свой - говорить = ('озарять', 0.5926368236541748) 

усадьба + суходол - коза = ('луневый', 0.5771392583847046) 

дом + пётр - сказать = ('губернский', 0.6742419004440308) 

поле + петрович - пашка = ('уваров', 0.5953262448310852) 

день + дом - гимназист = ('город', 0.6235319972038269) 

свой + наталья - стать = ('девичий', 0.5826830863952637) 

небо + барышня - старик = ('чувствовать', 0.7123276591300964) 

чёрный + говорить - федот = ('дух', 0.49777066707611084) 

собака + наташка - свой = ('смеяться', 0.6194528341293335) 

глаз + дедушка - рука = ('жутко', 0.5231676697731018) 

лес + глаз - голова = ('простор', 0.583296000957489) 

яблоко + герваська - мужик = ('холодный', 0.5254985690116882) 

пора + стать - взять = ('близкий', 0.5820208191871643) 

жить + голова - нога = ('то', 0.5677953958511353) 

помнить + тёмный - лицо = ('знойный', 0.5575349926948547) 

шалаш + день - самый = ('ухаб', 0.6220269203186035) 

голова + сад - прямо = ('ветерок', 0.608254611492157) 

листва +